In [2]:
from data.data_pipelines import etl_dowaward_giordano

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
df = etl_dowaward_giordano()

In [ ]:
def calculate_correlation(df):
    # TODO aveage relative correlation calculation
    pass

def calculate_ATR(df):
    pass

def rank_signals(df, columns_to_rank):
    pass